In [26]:
import pandas as pd

In [27]:
audience_count = pd.read_csv(f"audiences_per_month.csv")
audience_count = audience_count[[col for col in audience_count.columns if "Unnamed" not in col]]
conversion_count = pd.read_csv(f"conversions_count_2025-01-20_2025-07-25.csv")
conversion_count = conversion_count[[col for col in conversion_count.columns if "Unnamed" not in col]]
event_count = pd.read_csv(f"PRD 2581 Pricing Count Sessions.csv")
event_count = event_count[[col for col in event_count.columns if "Unnamed" not in col]]
targeted_profiles = pd.read_parquet("targeted_profiles_per_audience_2025-01-21_2025-07-26.parquet")
targeted_profiles = targeted_profiles[[col for col in targeted_profiles.columns if "Unnamed" not in col]]


In [28]:
event_count.info()
event_count.head()

In [29]:
event_count["month_orig"] = event_count["month"]
import datetime

def convert_month(s):
    if len(s) == 6:
        year = int(s[:4])
        month = int(s[4:])
        return f"{year}-{month:02}"
    else:
        return s

event_count["month"] = event_count["month_orig"].apply(lambda x: convert_month(str(x)))
event_count = event_count.drop(columns=["month_orig"])
event_count

In [30]:
audience_count.info()
audience_count.head()

In [31]:
conversion_count.info()
conversion_count.head()

## Preprocess targeted profiles

In [42]:
targeted_profiles["month"] = pd.to_datetime(targeted_profiles["date"], format="mixed").dt.strftime("%Y-%m")
targeted_profiles["month"] = targeted_profiles["month"].astype("string")
targeted_profiles = targeted_profiles[targeted_profiles["audience_id"].str.startswith("conversion_predictions_")==False]
targeted_profiles

In [43]:
targeted_profiles_by_month = targeted_profiles.groupby(["account","month"])["unique_anonymous_ids"].sum().reset_index().rename(columns={"unique_anonymous_ids": "targeted_anonymousIds"})
targeted_profiles_by_month

In [44]:
targeted_profiles.groupby(["account","month"])["audience_id"].nunique().reset_index()

# Merge data

In [45]:
merged = pd.merge(
    audience_count[audience_count["month"]>="2025-01"], 
    conversion_count,
    how="outer",
    on=["account","month"]
)
merged = pd.merge(
    merged,
    event_count,
    how="outer",
    on=["account","month"]
)
merged = pd.merge(
    merged,
    targeted_profiles_by_month,
    how="outer",
    on=["account","month"]
)
print(len(audience_count[audience_count["month"]>="2025-01"]), len(conversion_count), len(targeted_profiles_by_month),len(merged))
merged = merged.sort_values(by=["account","month"])
merged = merged.drop_duplicates()
merged = merged.dropna(subset=["name"])
merged = merged[merged["month"]!="2025-07"].reset_index(drop=True)
print(len(audience_count[audience_count["month"]>="2025-01"]), len(conversion_count), len(targeted_profiles_by_month),len(merged))
merged

In [46]:
merged.to_csv("pricing_data.csv",index=False)

In [36]:
1354001-45000*30